# Importación de los Datos

In [1]:
# Importamos librerias y definimos parámetros
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.copy_on_write = True
pd.set_option('display.max_rows', 10)

In [2]:
# Leer el archivo excel en su totalidad
excel_file = 'C:/Información/Master Big Data n Analytics/TFM/2022-2023 Joan Miró_solo_accesos.xlsx'
excel_completo = pd.read_excel(excel_file, sheet_name=None)

In [3]:
# Crear un diccionario para almacenar cada pestaña en una variable diferente
DF = {}

# Recorrer todas las pestañas y almacenarlas en el diccionario
for nombre_pestaña, df in excel_completo.items():
    DF[nombre_pestaña] = df

# Manipulación de datos

# Tabla de Accesos

In [4]:
# Vinculamos el DataFrame de Tabla de Accesos a la variable accesos
accesos = DF['C1329 Tabla_accesos']

In [5]:
# Rellenamos todas las celdas vacías con 0 para poder operar con el modelo en el futuro
accesos.replace(' ', np.nan, inplace = True)
accesos.fillna(0, inplace = True)

C:\Users\Joan\AppData\Local\Temp\ipykernel_18628\2738407397.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  accesos.replace(' ', np.nan, inplace = True)


In [6]:
# Convertimos los nombres de las columnas menusales a formato fecha para poder operar con las mismas
nombres_columna_mes = accesos.columns[1:]
mes_fecha = datetime(2022, 1, 1)

for mes in nombres_columna_mes:
    accesos.rename(columns={mes: mes_fecha}, inplace=True)
    mes_fecha = pd.Timestamp(mes_fecha) + pd.DateOffset(months=1)

In [7]:
# Pasamos los meses de todas las columnas a una única columna con los 24 meses para cada IdPersona
accesos = pd.melt(accesos, id_vars=['IdPersona'], var_name='Mes', value_name='Accesos')
accesos = accesos.sort_values(by=['IdPersona', 'Mes'])

In [8]:
# Añadimos 2 columnas de lags y eliminamos las filas con valores nulos
accesos['Accesos-1'] = accesos.groupby('IdPersona')['Accesos'].shift(1)
accesos['Accesos-2'] = accesos.groupby('IdPersona')['Accesos'].shift(2)
accesos.dropna(inplace = True)

In [9]:
# usuarios = [15, 20, 39, 56, 126, 149, 151, 161, 55546]
# accesos = accesos[accesos['IdPersona'].isin(usuarios)]

# Creación tabla información de cliente y tablas de variables menusales

In [10]:
# Juntamos las 24 tablas mensuales y creamos un DataFrame
dfs_mensuales = [DF['Ene_22'], DF['Feb_22'], DF['Mar_22'], DF['Abr_22'], DF['May_22'], DF['Jun_22'], DF['Jul_22'], DF['Ago_22'], DF['Sep_22'], DF['Oct_22'], DF['Nov_22'], DF['Dic_22'], DF['Ene_23'], DF['Feb_23'], DF['Mar_23'], DF['Abr_23'], DF['May_23'], DF['Jun_23'], DF['Jul_23'], DF['Ago_23'], DF['Sep_23'], DF['Oct_23'], DF['Nov_23'], DF['Dic_23']]
tablas_mensuales = pd.concat(dfs_mensuales, ignore_index = True)

# Creamos 2 DataFrames
# Información de clientes:
info_clientes = tablas_mensuales.drop_duplicates(subset = ['IdPersona'])
info_clientes = info_clientes[['IdPersona', 'Edad', 'Sexo', 'EsAbonado']].copy()

# Se eliminan las filas que presentan valores de edad incorrectos
info_clientes['Edad'] = pd.to_numeric(info_clientes['Edad'], errors='coerce')
info_clientes = info_clientes[info_clientes['Edad'].notna()]

# Información variable en el tiempo (altas, bajas, tipo de abono):
Info_fechas = tablas_mensuales.drop_duplicates(subset = ['IdPersona', 'FechaAlta', 'FechaBajaAbono'])
altas_bajas_abonos = Info_fechas[['IdPersona', 'FechaAlta', 'FechaBajaAbono', 'BajaPorCambio', 'TipoAbono']].copy()

# Tabla Altas-Bajas-Accesos

In [11]:
# Convertimos las columnas FechaAlta y FechaBajaAbono a tipo fecha para poder operar con la misma
altas_bajas_abonos['FechaBajaAbono'] = pd.to_datetime(altas_bajas_abonos['FechaBajaAbono'], format='%d/%m/%Y', errors = 'coerce')
altas_bajas_abonos['FechaAlta'] = pd.to_datetime(altas_bajas_abonos['FechaAlta'], format='%d/%m/%Y', errors = 'coerce')
# altas_bajas['FechaBajaAbono'] = altas_bajas['FechaBajaAbono'].dt.date() # (Supuestamente para convertir DateTime to Date, pero no funciona con series) - No se encuengtra la manera de pasar a Date

# Convertimos la columna BajaPorCambio en booleana
altas_bajas_abonos['BajaPorCambio'] = altas_bajas_abonos['BajaPorCambio'].map({'Sí': True, 'No': False})

In [12]:
# Rellenamos las fechas vacías de FechasBaja Abono (usuarios que no se dan de baja) con una fecha futura para poder operar más adelante
Fecha_sin_baja = pd.to_datetime('2024-01-02')
altas_bajas_abonos['FechaBajaAbono'] = altas_bajas_abonos['FechaBajaAbono'].fillna(Fecha_sin_baja)

In [13]:
# altas_bajas_abonos = altas_bajas_abonos[altas_bajas_abonos['IdPersona'].isin(usuarios)]
# altas_bajas_abonos.drop_duplicates(inplace= True)

In [14]:
# Juntamos ambas tablas en la variable IdPersona para que se repitan todas esas filas que para un mismo ID presentan variedad de datos en altas, bajas y abonos
accesos_altas_bajas = pd.merge(accesos, altas_bajas_abonos, on='IdPersona', how='inner')
accesos_altas_bajas.drop_duplicates(inplace= True)

# Unión de accesos con altas-bajas-accesos y limpieza de datos

In [15]:
# Creamos una columna booleana que indica para cada mes de accesos, si el usuario esta dado de alta o no (si la FechaAlta es igual o inferior al último día del mes y la FechaBajaAbono es superior al primer día del mes)
accesos_altas_bajas['EsSocio'] = ((accesos_altas_bajas['Mes'] + pd.DateOffset(months=1) - pd.DateOffset(days=1)) >= accesos_altas_bajas['FechaAlta']) & (accesos_altas_bajas['Mes'] < accesos_altas_bajas['FechaBajaAbono'])

# Descartamos todas las filas en que los usuarios no están dados de alta
accesos_altas_bajas = accesos_altas_bajas[accesos_altas_bajas['EsSocio']].copy()

In [16]:
# Se crea una nueva columna booleana que indica la baja en el mes que se da de baja (Se contempla desde el día 2 del més hasta el día 1 del mes siguiente)
accesos_altas_bajas['Baja'] = (accesos_altas_bajas['Mes'] < accesos_altas_bajas['FechaBajaAbono']) & (accesos_altas_bajas['FechaBajaAbono'] <= (accesos_altas_bajas['Mes'] + pd.DateOffset(months=1))) & (accesos_altas_bajas['BajaPorCambio'] == False)

# Cálculo de antiguedad de los miembros

In [17]:
# Se crea un nuevo DataFrame con el primer registro de cada usuario
IdPersonas = accesos_altas_bajas.groupby('IdPersona').first().reset_index()

# Se calcula la antiguedad inicial de cada miembro
def Antiguedad_Inicial(fila):
    Diferencia = relativedelta(fila['Mes'], fila['FechaAlta'])
    return Diferencia.years * 12 + Diferencia.months

IdPersonas['AntiguedadInicial'] = IdPersonas.apply(Antiguedad_Inicial, axis=1)

In [18]:
# Unimos el DataFrame princiapl con el de la antiguedad inicial de cada usuario
accesos_altas_bajas = accesos_altas_bajas.merge(IdPersonas[['IdPersona', 'AntiguedadInicial']], on='IdPersona', how='inner')

In [19]:
# Se calcula la antiguedad para cada mes partiendo de la antiguedad inicial previamente calculada, reiniciando en caso de baja
def Antiguedad_Miembro(Usuario):
    Antiguedad = Usuario['AntiguedadInicial'].iloc[0]
    Lista_Antiguedades = []
    for Baja in Usuario['Baja']:
        Lista_Antiguedades.append(Antiguedad)
        if Baja:
            Antiguedad = 0
        else:
            Antiguedad = Antiguedad + 1
    Usuario['MesesAntiguedad'] = Lista_Antiguedades
    return Usuario

accesos_altas_bajas = accesos_altas_bajas.groupby('IdPersona').apply(Antiguedad_Miembro).reset_index(drop=True)

C:\Users\Joan\AppData\Local\Temp\ipykernel_18628\4217161863.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accesos_altas_bajas = accesos_altas_bajas.groupby('IdPersona').apply(Antiguedad_Miembro).reset_index(drop=True)


# Mes de Alta

In [20]:
# Creamos una nueva columna booleana con las bajas desfasadas 1 registro y realizamos la suma acumulativas de dichos valores por cada usuario
accesos_altas_bajas['Realta'] = accesos_altas_bajas.groupby('IdPersona')['Baja'].shift(1).fillna(False).astype(bool)
accesos_altas_bajas['Realta'] = accesos_altas_bajas.groupby('IdPersona')['Realta'].cumsum()

# Se agrupan en conjuntos de usuarios y valor acumulado para diferenciar los periodos de los diferentes abonos y obtener dicho valor mensual
accesos_altas_bajas['FechaAltaPorGrupoAbono'] = accesos_altas_bajas.groupby(['IdPersona', 'Realta'])['FechaAlta'].transform('first')
accesos_altas_bajas['MesAlta'] = accesos_altas_bajas['FechaAltaPorGrupoAbono'].dt.month

C:\Users\Joan\AppData\Local\Temp\ipykernel_18628\3142005160.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  accesos_altas_bajas['Realta'] = accesos_altas_bajas.groupby('IdPersona')['Baja'].shift(1).fillna(False).astype(bool)


# Cálculo de accesos trimestrales

In [21]:
# Se realiza el sumatorio de los 3 meses de accesos por registro
accesos_altas_bajas['AccesosTrimestrales'] = accesos_altas_bajas['Accesos'] + accesos_altas_bajas['Accesos-1'] + accesos_altas_bajas['Accesos-2']

# Unión con la información de los usuarios

In [22]:
# Se une la tabla principal con la tabla de información de usuario
Tabla_semicompleta = pd.merge(accesos_altas_bajas, info_clientes, on='IdPersona', how='inner')

# Limpieza de las columnas sobrantes

In [23]:
Tabla_completa = Tabla_semicompleta.drop(columns=['FechaAlta', 'FechaBajaAbono', 'BajaPorCambio', 'EsSocio', 'AntiguedadInicial', 'Realta', 'FechaAltaPorGrupoAbono'])

# FechaAlta

In [24]:
Tabla_completa = Tabla_completa[['IdPersona', 'Mes', 'Edad', 'Sexo', 'EsAbonado', 'TipoAbono', 'MesAlta', 'Accesos', 'Accesos-1', 'Accesos-2', 'AccesosTrimestrales', 'MesesAntiguedad', 'Baja']]

In [25]:
Tabla_completa

,IdPersona,Mes,Edad,Sexo,EsAbonado,TipoAbono,MesAlta,Accesos,Accesos-1,Accesos-2,AccesosTrimestrales,MesesAntiguedad,Baja
0,15,2022-03-01 00:00:00,66.0,Mujer,Sí,ABONAMENT GENERAL,10,5.0,7.0,1.0,13.0,41,False
1,15,2022-04-01 00:00:00,66.0,Mujer,Sí,ABONAMENT GENERAL,10,4.0,5.0,7.0,16.0,42,False
2,15,2022-05-01 00:00:00,66.0,Mujer,Sí,ABONAMENT GENERAL,10,5.0,4.0,5.0,14.0,43,False
3,15,2022-06-01 00:00:00,66.0,Mujer,Sí,ABONAMENT GENERAL,10,3.0,5.0,4.0,12.0,44,False
4,15,2022-07-01 00:00:00,66.0,Mujer,Sí,ABONAMENT GENERAL,10,0.0,3.0,5.0,8.0,45,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115824,56637,2023-08-01 00:00:00,37.0,Hombre,Sí,- QUOTA FREE,1,0.0,0.0,0.0,0.0,7,False
115825,56637,2023-09-01 00:00:00,37.0,Hombre,Sí,- QUOTA FREE,1,0.0,0.0,0.0,0.0,8,False
115826,56637,2023-10-01 00:00:00,37.0,Hombre,Sí,- QUOTA FREE,1,0.0,0.0,0.0,0.0,9,False
115827,56637,2023-11-01 00:00:00,37.0,Hombre,Sí,- QUOTA FREE,1,0.0,0.0,0.0,0.0,10,False


# Conversión de las variables categóricas

In [26]:
# Asociamos la tabla a una variable diferentes
Tabla_modelos = Tabla_completa

In [27]:
# Se cambia el nombre de la columna 'Sexo'
Tabla_modelos.rename(columns={'Sexo': 'Genero_masculino'}, inplace=True)

# Cambiamos los valores str a booleanos
Tabla_modelos['Genero_masculino'] = Tabla_modelos['Genero_masculino'].map({'Hombre': True, 'Mujer': False})
Tabla_modelos['EsAbonado'] = Tabla_modelos['EsAbonado'].map({'Sí': True, 'No': False})

In [28]:
# Usamos get_dummies para la columna TipoAbono y el mes de alta
Tabla_modelos = pd.get_dummies(Tabla_modelos, columns = ['TipoAbono', 'MesAlta'])

In [29]:
Tabla_modelos

,IdPersona,Mes,Edad,Genero_masculino,EsAbonado,Accesos,Accesos-1,Accesos-2,AccesosTrimestrales,MesesAntiguedad,...,MesAlta_3,MesAlta_4,MesAlta_5,MesAlta_6,MesAlta_7,MesAlta_8,MesAlta_9,MesAlta_10,MesAlta_11,MesAlta_12
0,15,2022-03-01 00:00:00,66.0,False,True,5.0,7.0,1.0,13.0,41,...,False,False,False,False,False,False,False,True,False,False
1,15,2022-04-01 00:00:00,66.0,False,True,4.0,5.0,7.0,16.0,42,...,False,False,False,False,False,False,False,True,False,False
2,15,2022-05-01 00:00:00,66.0,False,True,5.0,4.0,5.0,14.0,43,...,False,False,False,False,False,False,False,True,False,False
3,15,2022-06-01 00:00:00,66.0,False,True,3.0,5.0,4.0,12.0,44,...,False,False,False,False,False,False,False,True,False,False
4,15,2022-07-01 00:00:00,66.0,False,True,0.0,3.0,5.0,8.0,45,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115824,56637,2023-08-01 00:00:00,37.0,True,True,0.0,0.0,0.0,0.0,7,...,False,False,False,False,False,False,False,False,False,False
115825,56637,2023-09-01 00:00:00,37.0,True,True,0.0,0.0,0.0,0.0,8,...,False,False,False,False,False,False,False,False,False,False
115826,56637,2023-10-01 00:00:00,37.0,True,True,0.0,0.0,0.0,0.0,9,...,False,False,False,False,False,False,False,False,False,False
115827,56637,2023-11-01 00:00:00,37.0,True,True,0.0,0.0,0.0,0.0,10,...,False,False,False,False,False,False,False,False,False,False


# Exportamos el DataFrame a Excel

In [30]:
ruta_archivo = 'C:/Información/Master Big Data n Analytics/TFM/DatosParaModelo.xlsx'

In [31]:
# Tabla_modelos.to_excel(ruta_archivo, index=False, sheet_name='DataFrame')